# Importacion de Librerias básicas

Con estas librerias se puede hacer un rápido análisis de datos previo a subirlos a la BDD, donde se puede hacer más rápido una limpieza de datos, revisar validaciones y realizar gráficos mamalones :D

El resto de codigo solo es personalizacion 

In [41]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_columns = None


sns.set_theme(style="whitegrid")
sns.set_style("ticks", {"font.family": "Times New Roman"})

colores = plt.cm.tab20.colors
font = {'family': 'Times New Roman', 'size': 14, 'weight': 'normal'}
plt.rcParams['font.family'] = font['family']
plt.rcParams['font.size'] = font['size']
plt.rcParams['font.weight'] = font['weight']

Cargamos el CSV

In [42]:
path = r'D:\Quiz Monse\DataSet\restaurantes.csv'
df_original = pd.read_csv(path)
df_original.head(1)

,id,rating,name,site,email,phone,street,city,state,lat,lng
0,851f799f-0852-439e-b9b2-df92c43e7672,1,"Barajas, Bahena and Kano",https://federico.com,Anita_Mata71@hotmail.com,534 814 204,82247 Mariano Entrada,Mérida Alfredotown,Durango,19.440057,-99.127047


In [43]:
df = df_original.copy() # Copia de reslpaldo
df.shape

(100, 11)

La bdd tiene más de 30 registros sufieciente para que tenga relevancia estadistica conforme al **Teorema del Límite Central (TLC)**


Analizamos el tamaño de los string para crear la bdd

In [44]:
def CaracteresContador (df:pd.DataFrame)->None:
    str_columns = df.select_dtypes(include=['object']).columns
    max_lengths = {}
    min_lengths = {}

    for col in str_columns:
        lengths = df[col].str.len()
        max_lengths[col] = lengths.max()
        min_lengths[col] = lengths.min()
    print(f"Longitudes máximas: {max_lengths}")
    print(f"Longitudes mínimas: {min_lengths}")

In [45]:
CaracteresContador (df)

Longitudes máximas: {'id': 36, 'name': 32, 'site': 29, 'email': 38, 'phone': 12, 'street': 27, 'city': 36, 'state': 21}
Longitudes mínimas: {'id': 36, 'name': 10, 'site': 15, 'email': 17, 'phone': 9, 'street': 15, 'city': 8, 'state': 6}


podemos llegar a las concluir que 
- *id* es un rowguid
- *name* podemos agregar 40 de longitud máxima, ya que damos un margen de referencia para posibles nombres que sobrepasen el valor máximo
- *site* podemos agregar 40 de longitud máxima
- *email* podemos agregar 45 de longitud máxima
- *street* podemos agregar 35 de longitud máxima
- *city* podemos agregar 50 de longitud máxima
- *state* podemos agregar 30 de longitud máxima

Para phone vamos a realizar una limpieza de datos

In [46]:
df['phone'] = df['phone'].str.replace(r'\D', '', regex=True)
CaracteresContador (df)

Longitudes máximas: {'id': 36, 'name': 32, 'site': 29, 'email': 38, 'phone': 10, 'street': 27, 'city': 36, 'state': 21}
Longitudes mínimas: {'id': 36, 'name': 10, 'site': 15, 'email': 17, 'phone': 9, 'street': 15, 'city': 8, 'state': 6}


In [47]:
print(f'Para phone = 9, tenemos {df[df["phone"].apply(lambda x: len(x) == 9)].shape[0]} resultados')
print(f'Para phone = 10, tenemos {df[df["phone"].apply(lambda x: len(x) == 10)].shape[0]} resultados')


Para phone = 9, tenemos 71 resultados
Para phone = 10, tenemos 29 resultados


In [48]:
df[df["phone"].apply(lambda x: len(x) == 9)][["phone","state"]]

,phone,state
0,534814204,Durango
1,570746998,Hidalgo
4,559867074,Colima
5,571744718,Michoacan
6,584212054,San Luis Potosí
...,...,...
94,552972577,Baja California Sur
95,520411112,Chihuahua
97,500281343,Tamaulipas
98,590666046,Michoacan


Por la normativa, los números telefónicos están regidos por el Instituto Federal de Telecomunicaciones (IFT). A partir del 3 de agosto de 2019, [referencia][1]. 

Los números telefónicos deben ser de 10 dígitos. Dado que esto es un quiz, asumiré que es parte de la limpieza de datos, por lo tanto, se agregará un '5' al inicio de cada número. Además, estos números no coinciden con las claves lada de los estados, pero en producción contemplaría la opción de eliminarlos, no insertalos o corregirlos dependiendo de las instrucciones del líder del proyecto, ya que no cumplen con la normativa, a pesar de ser la mayoría de la base de datos  ([referencia][2] columna NIR) >:( 


[1]:(https://www.ift.org.mx/comunicacion-y-medios/comunicados-ift/es/partir-del-3-de-agosto-mexico-tendra-una-nueva-forma-de-marcacion-telefonica-comunicado-342019-16-de#:~:text=Para%20las%20llamadas%20a%20n%C3%BAmeros,se%20eliminar%C3%A1%20el%20prefijo%2001.)
[2]:(https://sns.ift.org.mx:8081/sns-frontend/planes-numeracion/descarga-publica.xhtml)

In [49]:
df.loc[df["phone"].apply(lambda x: len(x) == 9), "phone"] = "5" + df["phone"]
CaracteresContador (df)

Longitudes máximas: {'id': 36, 'name': 32, 'site': 29, 'email': 38, 'phone': 10, 'street': 27, 'city': 36, 'state': 21}
Longitudes mínimas: {'id': 36, 'name': 10, 'site': 15, 'email': 17, 'phone': 10, 'street': 15, 'city': 8, 'state': 6}


## Validacion de email

In [50]:
regex_email = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
invalid_emails = df[~df["email"].str.match(regex_email, na=False)]

print("Correos electrónicos inválidos:")
print(invalid_emails)

Correos electrónicos inválidos:
Empty DataFrame
Columns: [id, rating, name, site, email, phone, street, city, state, lat, lng]
Index: []


Apesar de que no las detecte hay acentos y espacios por ello vamos a solucionarlo

In [51]:
import unidecode

def make_url_valid(url):
    url = unidecode.unidecode(url)
    url = url.replace(' ', '-')
    return url

df["email"] = df["email"].apply(make_url_valid)

Se observa que todos los correos son validos :D 

## Validar de URL

In [52]:
regex_email = r'^(https?:\/\/)?([\da-z.-]+)\.([a-z.]{2,6})([\/\w .-]*)*\/?$'
invalid_emails = df[~df["site"].str.match(regex_email, na=False)]

print(f"URL inválidos: {invalid_emails.shape[0]}")
invalid_emails.head(1)

URL inválidos: 26


,id,rating,name,site,email,phone,street,city,state,lat,lng
4,2b8f5a44-1e8b-44ec-9b25-0edc5b64b7e6,3,"Hurtado, Rolón and Segovia",https://elías.org,RosaMara_Figueroa@corpfolder.com,5559867074,039 Susana Polígono,Marco Antonioland,Colima,19.433498,-99.128514


In [53]:
df["site"] = df["site"].apply(make_url_valid)
invalid_emails = df[~df["site"].str.match(regex_email, na=False)]

print(f"URL inválidos: {invalid_emails.shape[0]}")

URL inválidos: 0


## Validar Rating

In [54]:
df["rating"].unique()

array([1, 0, 3, 2, 4], dtype=int64)

## Validar coordenadas

In [55]:
filtered_df = df[df[['lat', 'lng']].isna().any(axis=1)]
filtered_df

,id,rating,name,site,email,phone,street,city,state,lat,lng


In [56]:
df.head(1)

,id,rating,name,site,email,phone,street,city,state,lat,lng
0,851f799f-0852-439e-b9b2-df92c43e7672,1,"Barajas, Bahena and Kano",https://federico.com,Anita_Mata71@hotmail.com,5534814204,82247 Mariano Entrada,Mérida Alfredotown,Durango,19.440057,-99.127047


# Query
Dado que el DataSet, ya ha sido revisado y validado se procedera a insertarlo en la base de datos, dado que se puede realizar con pandas directamente (esto es muy lento con python y sin el conocimoiento puede dañar la bdd de producción, admirable quien lo haga así), guardarlo en un formato plano como csv, json, txt, entre otros y cargarlo desde la interfaz de la bdd. Prefiero crear un query y controlar la inserción.

**Nota:** son muy pocos datos (100 filas) no superan el limite de una insersion masiva (1000 filas) no es necesario modificar el limite (permisos de admin) o el codigo (para cualquier usuario)

In [57]:
df.columns

Index(['id', 'rating', 'name', 'site', 'email', 'phone', 'street', 'city',
       'state', 'lat', 'lng'],
      dtype='object')

In [58]:
df["email"].dtypes

dtype('O')

In [59]:
print ("""
DECLARE
    @ErrorMessage NVARCHAR(4000),
    @ErrorSeverity INT,
    @ErrorState INT

BEGIN TRY
    BEGIN TRANSACTION
       INSERT INTO Restaurant.Restaurants (Id, Rating, [Name], WebSite, Email, Phone, Street, City, [State], Lat, Lng)
       VALUES 
""", end="")
for _,row in df.iterrows():
    print("\t\t(",end="")
    for columna in df.columns:
        if pd.api.types.is_string_dtype(df[columna]):
            print (f"'{row[columna]}',",end="")
        elif columna == "lng": # por ser la ultima columna
            print (f"{row[columna]}),")
        else:
             print (f"{row[columna]},",end="")
print("""
    COMMIT TRANSACTION;
END TRY
BEGIN CATCH
    SELECT 
        @ErrorMessage = ERROR_MESSAGE(),
        @ErrorSeverity = ERROR_SEVERITY(),
        @ErrorState = ERROR_STATE();

        SET @ErrorMessage = 'Error en el procedimiento almacenado Insertar Certificado: ' + @ErrorMessage +
                    '. Error en la línea: ' + CAST(ERROR_LINE() AS NVARCHAR(10))
    RAISERROR (@ErrorMessage, @ErrorSeverity, @ErrorState);
    ROLLBACK TRANSACTION;
END CATCH
""")


DECLARE
    @ErrorMessage NVARCHAR(4000),
    @ErrorSeverity INT,
    @ErrorState INT

BEGIN TRY
    BEGIN TRANSACTION
       INSERT INTO Restaurant.Restaurants (Id, Rating, [Name], WebSite, Email, Phone, Street, City, [State], Lat, Lng)
       VALUES 
		('851f799f-0852-439e-b9b2-df92c43e7672',1,'Barajas, Bahena and Kano','https://federico.com','Anita_Mata71@hotmail.com','5534814204','82247 Mariano Entrada','Mérida Alfredotown','Durango',19.4400570537131,-99.1270470974249),
		('4e17896d-a26f-44ae-a8a4-5fbd5cde79b0',0,'Hernández - Lira','http://graciela.com.mx','Brandon_Vigil@hotmail.com','5570746998','93725 Erick Arroyo','Mateofurt','Hidalgo',19.437904276995,-99.1286576775023),
		('c0ffd058-e773-47f1-974b-42d41cb555bf',3,'Rendón - Elizondo','https://cristina.mx','Hugo.Casanova49@gmail.com','5866337812','5518 Monserrat Explanada','Chignahuapan María','Sinaloa',19.4360705910348,-99.1297865731994),
		('c29082c4-4352-4517-9b4b-c45f86fc1830',2,'Nájera - Chávez','https://pedro.gob.mx','Carl

Esto generara un error, pero se soluciona eliminando la ultima coma de la ultima fila